<a href="https://colab.research.google.com/github/esis-a-i/Data-Engineering-Arenadata/blob/main/Seminars/%D0%A1%D0%B5%D0%BC%D0%B8%D0%BD%D0%B0%D1%80_W3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Работа с HDFS: создание директории, загрузка файлов в HDFS

Семинар 2. Основные концепции

Елена Дорофеева. Аренадата Софтвер

## Цель семинара

Познакомить участников с основными командами для работы с HDFS в Hadoop, научить их создавать директории, загружать файлы и управлять ими в распределённой файловой системе.

## План семинара


1.   Введение в HDFS:
  
  1.1. Подключение к HDFS: подключение к кластеру Hadoop и проверка доступа к HDFS.
      
  1.2. Разрешения в HDFS.
      
  1.3. Основные команды HDFS: обзор команд для работы с HDFS (hdfs dfs и hadoop fs).
2.   Работа с директориями в HDFS:
  
  2.1. Создание директории.
  
  2.2. Просмотр содержимого директории.
3. Команды скачивания и загрузки файлов:
  
  3.1. Выгрузка файлов из HDFS.
  
  3.2. Загрузка файлов в HDFS.
4. Команды для чтения файлов в HDFS.
5. Управление данными в HDFS:
  
  5.1. Перемещение и копирование файлов.
  
  5.2. Удаление файлов и директорий.
6. Аналитика утилизации хранилища


## 1. Введение в HDFS
### 1.1. Подключение к HDFS: подключение к кластеру Hadoop и проверка доступа к HDFS

Чтобы работать в Hadoop, нужно стать пользователем Hadoop. Им можно стать с помощью создания домашней директории и наделения твоего пользователя полномочиями на эту директорию.

Это делается двумя командами: mkdir и chown. Это обязательное действие, если ты хочешь работать в&nbsp;Hadoop.

sudo -u hdfs — синтаксис команды, чтобы создать первоначальную директорию и наделить её правами.
Для этого нам нужны привилегии суперпользователя.

Cоздаём домашнюю директорию:

In [ ]:
sudo -u hdfs hadoop fs -mkdir /user/admin
sudo -u hdfs hadoop fs -chown admin:admin /user/admin

Наделяем твоего пользователя полномочиями на эту директорию:

In [ ]:
sudo -u hdfs hadoop fs -chown admin:admin /user/admin

Что будет, если у тебя не будет домашней директории:
1. Ты не сможешь создавать директории в HDFS (будет выведена ошибка, например:
mkdir: Permission denied: user=teacher_1, access=WRITE, inode="/":hdfs:hadoop:drwxr-xr-x).
2. Ты не сможешь запускать задачи MapReduce на вычисление, так как у YARN есть обязательное условие, чтобы была домашняя директория. Когда мы из-под пользователя запускаем задачу MapReduce, она сохраняет промежуточные результаты в твою домашнюю директорию.

### 1.2. Разрешения в HDFS

Каждому файлу и директории в HDFS присваивается владелец (owner), группа (group) и разрешения (permissions).
Разрешения включают три уровня доступа:

*   чтение (read);
*   запись (write);
*   выполнение (execute).

Разрешения задаются для трёх категорий пользователей: владельца (owner), группы (group) и других (others).

Посмотрим на разрешения на файлы и директории в корневой директории, а также на их владельцев и&nbsp;группы:

In [ ]:
!hadoop fs -ls /

Found 9 items
drwxrwxrwx   - hdfs hadoop          0 2024-10-21 18:21 /ETL
drwxr-xr-x   - hdfs hadoop          0 2024-07-10 10:29 /apps
drwx------   - livy hadoop          0 2024-07-10 10:30 /livy-recovery
drwx------   - livy hadoop          0 2024-07-10 10:29 /livy-spark3-recovery
drwxrwxrwt   - yarn hadoop          0 2024-10-21 08:59 /logs
drwxr-xr-x   - hdfs hadoop          0 2024-07-10 10:19 /system
drwxrwxrwx   - hdfs hadoop          0 2024-10-29 05:44 /tmp
drwxr-xr-x   - hdfs hadoop          0 2024-07-12 11:48 /user
drwxr-xr-x   - hdfs hadoop          0 2024-07-10 10:28 /var


In [ ]:
!hdfs dfs -ls -R /user


drwxr-xr-x   - educator  educator           0 2024-07-11 07:32 /user/educator
drwx------   - hdfs      hadoop             0 2024-07-11 06:22 /user/hdfs
ls: Permission denied: user=educator, access=READ_EXECUTE, inode="/user/hdfs":hdfs:hadoop:drwx------
drwxr-xr-x   - mapred    hadoop             0 2024-07-10 10:19 /user/history
drwxrwx---   - mapred    hadoop             0 2024-07-10 12:19 /user/history/done
ls: Permission denied: user=educator, access=READ_EXECUTE, inode="/user/history/done":mapred:hadoop:drwxrwx---
drwxrwxrwt   - mapred    hadoop             0 2024-09-06 15:07 /user/history/done_intermediate
drwxrwx---   - hdfs      hadoop             0 2024-07-11 06:16 /user/history/done_intermediate/hdfs
ls: Permission denied: user=educator, access=READ_EXECUTE, inode="/user/history/done_intermediate/hdfs":hdfs:hadoop:drwxrwx---
drwxrwx---   - teacher_1 hadoop             0 2024-09-23 11:46 /user/history/done_intermediate/teacher_1
ls: Permission denied: user=educator, access=READ_

Разбор вывода:

*   Первая колонка показывает права доступа, например: -rw-r--r--, r — чтение, w — запись, x — выполнение.
*   Первая часть (например, rw-) относится к владельцу.
*   Вторая часть (r--) относится к группе.
*   Третья часть (r--) относится к другим пользователям.
*   Далее идёт владелец файла и группа.

Например, разрешение drwxrwxrwx означает, что всем можно всё.

Таким образом, файлы в HDFS можно читать, удалять, записывать, перезаписывать, но нельзя изменять. И нельзя запустить на выполнение файл.
Режимы доступа такие же, как и в Linux, но с «х» некоторые проблемы. А именно: для файлов «х» игнорируется, а для каталогов это разрешение даёт возможность проваливаться вниз каталога.

### 1.3. Основные команды HDFS: обзор команд для работы с HDFS (hdfs dfs и hadoop fs)

Для взаимодействия с HDFS используются команды, аналогичные стандартным UNIX-командам, через утилиты hdfs dfs и hadoop fs.
Команды hdfs dfs и hadoop fs практически идентичны в использовании и предлагают одни и те же функции.
hdfs dfs используется специально для работы с HDFS, в том числе поддерживает команды для администрирования.
hadoop fs является более универсальной командой, которая может работать с различными файловыми системами, поддерживаемыми Hadoop.

## 2. Работа с директориями в HDFS

### 2.1. Создание директории

Команда mkdir: создание новых директорий в HDFS.

Создаём директорию /user/yourname/testload:

In [ ]:
!hadoop fs -mkdir testload

Проверяем наличие созданной директории:

In [ ]:
!hadoop fs -ls

### 2.2. Просмотр содержимого директории

Команда ls: просмотр содержимого директории.

Проверяем содержимое директории /user/yourname/testload:

In [ ]:
!hadoop fs -ls testload

## 3. Команды скачивания и загрузки файлов

### 3.1. Выгрузка файлов из HDFS

Команда get: копирует файл из каталога HDFS в каталог локальной файловой системы.
https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/FileSystemShell.html#get

Загружаем в свой текущий рабочий каталог файл из HDFS:

In [ ]:
!hadoop fs -get /tmp/data.csv ~/data_from_hadoop.csv

Проверяем наличие файла в текущем рабочем каталоге:

In [ ]:
!ls

### 3.2. Загрузка файлов в HDFS

Команда put: загрузка файлов из локальной файловой системы в HDFS.
https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/FileSystemShell.html#put

Загружаем файл в HDFS, в ранее созданную директорию testload:

In [ ]:
!hadoop fs -put /home/teacher_1/data_from_hadoop.csv testload/data.csv

Проверяем наличие файла в HDFS, в директории testload:

In [ ]:
!hadoop fs -ls /user/teacher_1/testload

## 4. Команды для чтения файлов в HDFS

Команда cat выводит содержимое HDFS-файла в stdout (стандартный вывод на экран).

Посмотрим содержимое файла, находящегося в директории testload:

In [ ]:
!hadoop fs -cat /user/teacher_1/testload/data.csv

Команда head выводит первый килобайт файла в stdout (стандартный вывод на экран).
Подходит, когда достаточно вывести первые несколько строк файла, а не открывать полностью весь большой файл:

In [ ]:
!hadoop fs -head /user/teacher_1/testload/data.csv

Команда text принимает исходный файл и выводит файл в текстовом формате в stdout:

In [ ]:
!hadoop fs -text /user/teacher_1/testload/data.csv

## 5. Управление данными в HDFS

### 5.1. Перемещение и копирование файлов

Команда mv перемещает файлы из источника в место назначения в файловой системе HDFS. Позволяет указать несколько источников, в этом случае местом назначения должен быть каталог. Перемещение файлов между разными файловыми системами не допускается.
Возвращает 0 при успешной попытке и -1 при ошибке.

https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/FileSystemShell.html#mv

Создаём в HDFS новую директорию testload1:

In [ ]:
!hadoop fs -mkdir testload1

Проверяем наличие созданной директории:

In [ ]:
!hadoop fs -ls

Перемещаем файл data.csv из директории testload в новую директорию testload1:

In [ ]:
!hadoop fs -mv /user/teacher_1/testload/data.csv /user/teacher_1/testload1/data1.csv

Проверяем в директории testload1 наличие файла data1.csv:

In [ ]:
!hadoop fs -ls testload1

Проверяем отсутствие файла data.csv в директории testload (он должен переместиться):

In [ ]:
!hadoop fs -ls testload

Команда cp копирует файлы из источника в место назначения.
Команда позволяет использовать несколько источников, в этом случае местом назначения должен быть каталог.
https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/FileSystemShell.html#cp

Копируем файл data1.csv из директории testload1 в директорию testload:

In [ ]:
!hadoop fs -cp /user/teacher_1/testload1/data1.csv /user/teacher_1/testload/data.csv

Проверяем наличие файла data.csv в директории testload:

In [ ]:
!hadoop fs -ls testload

Также проверяем наличие файла data1.csv в директории testload1 (он также должен быть, так как это копирование, а не перемещение):

In [ ]:
!hadoop fs -ls testload1

### 5.2. Удаление файлов и директорий

Есть два способа удаления файлов и директорий в HDFS: с перемещением в корзину (удаление с&nbsp;возможностью восстановления) и без перемещения в корзину (необратимое удаление).

Удаление файлов с перемещением в корзину
В HDFS:
файлы по умолчанию удаляются в корзину, что позволяет восстановить их в случае необходимости.
Корзина в HDFS функционирует аналогично корзине в операционных системах — файлы временно перемещаются в специальную директорию .Trash, откуда они могут быть восстановлены до окончательного удаления.

Команда rm позволяет удалить файл/директорию:
https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/FileSystemShell.html#rm

Удаляем файл data1.csv, находящийся в директории testload1:

In [ ]:
!hadoop fs -rm /user/teacher_1/testload1/data1.csv

Проверяем, что файл удалился:

In [ ]:
!hadoop fs -ls testload1

Проверяем появление скрытого каталога .Trash и наличие в нём ранее удалённого файла data1.csv:

In [ ]:
!hadoop fs -lsr .Trash

Удаляем директорию testload1, используя в команде аргумент -R:

In [ ]:
!hadoop fs -rm -R /user/teacher_1/testload1

Примечание: если необходимо, файл можно восстановить из корзины, используя команду mv (перемещение из директории .Trash).

Удаление файлов без перемещения в корзину (окончательное удаление).

Для необратимого удаления файлов и директорий в HDFS без перемещения в корзину используется опция -skipTrash.
Это удобно, когда нужно немедленно освободить место или когда уверен, что данные больше не&nbsp;понадобятся.

Удаляем файл data.csv без перемещения в корзину (окончательное удаление):

In [ ]:
!hadoop fs -rm -skipTrash /user/teacher_1/testload/data.csv

Проверяем появление скрытого каталога .Trash и отсутствие в нём ранее удалённого файла data.csv:

In [ ]:
!hadoop fs -lsr .Trash

## 6. Аналитика утилизации хранилища

In [ ]:
import os
import subprocess
import pandas as pd

In [ ]:
cmd = 'hdfs dfs -ls -R /user/teacher_1/testload'
files = subprocess.check_output(cmd, shell=True)
files

b'-rw-r--r--   3 teacher_1 teacher_1    1475504 2024-09-06 11:02 /user/teacher_1/testload/data.csv\ndrwxr-xr-x   - teacher_1 teacher_1          0 2024-09-06 15:07 /user/teacher_1/testload/output\n-rw-r--r--   3 teacher_1 teacher_1          0 2024-09-06 15:07 /user/teacher_1/testload/output/_SUCCESS\n-rw-r--r--   3 teacher_1 teacher_1        552 2024-09-06 15:07 /user/teacher_1/testload/output/part-r-00000\n'

In [ ]:
all_rows = files.split()
rows = [[all_rows[j+i] for j in range(8)] for i in range(0,len(all_rows),8)]
tc = pd.DataFrame(rows, columns= ['access','col1','user','group','size','date','time','path'])
tc

,access,col1,user,group,size,date,time,path
0,b'-rw-r--r--',b'3',b'teacher_1',b'teacher_1',b'1475504',b'2024-09-06',b'11:02',b'/user/teacher_1/testload/data.csv'
1,b'drwxr-xr-x',b'-',b'teacher_1',b'teacher_1',b'0',b'2024-09-06',b'15:07',b'/user/teacher_1/testload/output'
2,b'-rw-r--r--',b'3',b'teacher_1',b'teacher_1',b'0',b'2024-09-06',b'15:07',b'/user/teacher_1/testload/output/_SUCCESS'
3,b'-rw-r--r--',b'3',b'teacher_1',b'teacher_1',b'552',b'2024-09-06',b'15:07',b'/user/teacher_1/testload/output/part-r-00000'


In [ ]:
for column in tc.columns:
    tc[column]=tc[column].apply(lambda x:x.decode("utf-8"))
tc['updated'] = pd.to_datetime(tc['date'] +' '+ tc['time'])
tc

,access,col1,user,group,size,date,time,path,updated
0,-rw-r--r--,3,teacher_1,teacher_1,1475504,2024-09-06,11:02,/user/teacher_1/testload/data.csv,2024-09-06 11:02:00
1,drwxr-xr-x,-,teacher_1,teacher_1,0,2024-09-06,15:07,/user/teacher_1/testload/output,2024-09-06 15:07:00
2,-rw-r--r--,3,teacher_1,teacher_1,0,2024-09-06,15:07,/user/teacher_1/testload/output/_SUCCESS,2024-09-06 15:07:00
3,-rw-r--r--,3,teacher_1,teacher_1,552,2024-09-06,15:07,/user/teacher_1/testload/output/part-r-00000,2024-09-06 15:07:00


In [ ]:
print(f"Утилизация хранилища: {round(tc['size'].astype(int).sum()/1024/1024,2)} MB")
print(f"Дата последнего обновления: {tc['updated'].max()}")

Утилизация хранилища: 1.41 MB
Дата последнего обновления: 2024-09-06 15:07:00
